In [21]:

import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    PromptTemplate,
)
from langchain.llms import OpenAI




In [22]:
GENERATIONS_PATH = "../datasets/baseline_zero_shot_gpt3_50_50_merged.csv"


data = pd.read_csv(GENERATIONS_PATH)
data.head()

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,predicted_label,label,error,similarity,confidence,whole_output,structured_explanation,structured_explanation_v2
0,0,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.793Z,2023-10-22T13:05:59.340Z,Are the following two products the same?\nProd...,False.,False,True,FN,30%,50%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
1,1,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:58.216Z,2023-10-22T13:05:58.791Z,Are the following two products the same?\nProd...,True,True,True,NONE,90%,80%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.825Z,2023-10-22T13:05:58.212Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.05"",...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
3,3,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:57.322Z,2023-10-22T13:05:57.823Z,Are the following two products the same?\nProd...,True,True,True,NONE,95%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,19817c50-cbc5-48b5-bf93-e6c45825d48b,gpt-3.5-turbo-0301,2023-10-22T13:05:56.925Z,2023-10-22T13:05:57.320Z,Are the following two products the same?\nProd...,True,True,True,NONE,100%,90%,NaN,"\n\n[{""attribute"":""brand"",""importance"":""0.2"",""...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."


In [23]:
list = []
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)
chain = LLMChain(llm=llm, prompt=PromptTemplate(
    input_variables=["error", "json"],
    template="""Please fix the JSON:
Error: {error}
JSON: {json}"""
)
)

for i, row in enumerate(data["structured_explanation"]):
    try:
        row_json = json.loads(row)
        list.append(row_json)
    except Exception as e1:
        # fixed_row = chain.run(json=row)
        # try:
        #     row_json = json.loads(fixed_row)
        #     list.append(row_json)
        # except Exception as e:
        #     print(e)
        #     print(i)
        #     print(row_json)
        try:
            fixed = row.replace(",\n"+ row.split("\n")[-1], "]")
            row_json = json.loads(fixed)
            list.append(row_json)
            print("fixed:" + str(i))
        except Exception as e2:
            print("-----")
            print(e2)
            # print(row)
            print(i)
            print("-----")
            # fixed_by_LLM = chain.run(error=e1, json=row)
            # try:
            #     row_json = json.loads(fixed_by_LLM)
            #     list.append(row_json)
            #     # print("It worked: " + fixed_by_LLM)
            # except Exception as e:
            #     print(e)
            #     print(i)
            #     print(fixed_by_LLM)
            #     print("------------------")
            #     print(row)
        


fixed:10
fixed:20
fixed:45
-----
Extra data: line 3 column 83 (char 84)
57
-----
fixed:61
-----
Expecting ',' delimiter: line 4 column 97 (char 179)
75
-----
fixed:85
fixed:131
fixed:133
-----
Extra data: line 3 column 77 (char 78)
134
-----
-----
Expecting ',' delimiter: line 4 column 70 (char 140)
144
-----
-----
Unterminated string starting at: line 3 column 871 (char 872)
158
-----
-----
Expecting ',' delimiter: line 4 column 102 (char 176)
166
-----
fixed:182


In [24]:
for item in list:
    print(item)

[{'attribute': 'brand', 'importance': '0.5', 'values': ["Traveler's Company", "Traveler's Company"]}, {'attribute': 'model', 'importance': '0.3', 'values': ['Notebook Refill - Regular Size', 'Notebook Accessories']}, {'attribute': 'color', 'importance': '0.2', 'values': ['missing', 'missing']}]
[{'attribute': 'brand', 'importance': '0.05', 'values': ['MINION', 'Maxxis']}, {'attribute': 'model', 'importance': '0.95', 'values': ['DHR II', 'Minion DHR II']}, {'attribute': 'color', 'importance': '0.00', 'values': ['missing', 'missing']}, {'attribute': 'size', 'importance': '0.00', 'values': ['29 X 2.30', '29"']}, {'attribute': 'casing', 'importance': '0.50', 'values': ['DD FOLD', 'DD/3C/TR']}, {'attribute': 'tread', 'importance': '0.50', 'values': ['Ramped and siped', 'missing']}, {'attribute': 'compound', 'importance': '0.50', 'values': ['3C MaxxTerra, 3C MaxxGrip, and Super Tacky', '3C Maxx Terra']}, {'attribute': 'trail width', 'importance': '0.50', 'values': ['Wide trail (WT)', 'missin